In [4]:
from confluent_kafka import Consumer, KafkaError
from pymongo import MongoClient
import json

# Configuración del consumidor de Kafka
kafka_config = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'instagram_consumer_group',  # Asegúrate de que este group.id es único para este consumidor
    'auto.offset.reset': 'earliest'
}

consumer = Consumer(kafka_config)
consumer.subscribe(['instagram_posts'])

# Configuración de la conexión a MongoDB
mongo_client = MongoClient('localhost', 27017)  # Cambia 'localhost' y '27017' si tu MongoDB está en otro host o puerto
db = mongo_client['instagram_data']  # 'instagram_data' es el nombre de la base de datos
collection = db['posts']  # 'posts' es el nombre de la colección donde se guardarán los datos

try:
    while True:
        message = consumer.poll(1.0)  # Ajusta el tiempo de espera si es necesario

        if message is None:
            continue
        if message.error():
            if message.error().code() == KafkaError._PARTITION_EOF:
                # Fin de la partición
                continue
            else:
                print("Error en el consumidor:", message.error())
                break

        # Convertir el mensaje de Kafka de bytes a JSON
        data = json.loads(message.value().decode('utf-8'))
        print('Recibido:', data)

        # Insertar el documento JSON en MongoDB
        collection.insert_one(data)

except KeyboardInterrupt:
    print("Detención manual del consumidor")

finally:
    consumer.close()  # Asegurarse de cerrar el consumidor de Kafka
    mongo_client.close()  # Asegurarse de cerrar la conexión a MongoDB


Recibido: {'username': 'harpistkt', 'likes': 1483, 'comments': 36, 'post_text': {'bit_flags': 0, 'created_at': 1728232479, 'created_at_utc': 1728232479, 'did_report_as_spam': False, 'is_ranked_comment': False, 'pk': '18053038066809900', 'share_enabled': False, 'content_type': 'comment', 'media_id': '3472987619326327204', 'status': 'Active', 'type': 1, 'user_id': '183894440', 'strong_id__': '18053038066809900', 'text': 'One of my favs of Joe Hisaishi to play on harp 🥰 \n\n#harp #ELECTRICharp #leverharp #howlsmovingcastle #merrygoroundoflife #anime #joehisaishi', 'user': {'pk': '183894440', 'pk_id': '183894440', 'id': '183894440', 'full_name': 'Kristan Toczko | Harpist', 'is_private': False, 'is_unpublished': False, 'strong_id__': '183894440', 'fbid_v2': '17841400673202873', 'username': 'harpistkt', 'is_verified': True, 'profile_pic_id': '2469111918284066427_183894440', 'profile_pic_url': 'https://scontent-iad3-1.cdninstagram.com/v/t51.2885-19/132031326_4739497119458014_15391273379338351